# Creating a Model for Jupyter Notebooks Using DSLModel

This notebook walks through the process of building a model to handle `.ipynb` files in Python using a custom `DSLModel` class structure.

## Step 1: Understanding the Notebook Structure

A Jupyter Notebook consists of several key components, such as metadata, cells, and outputs. These are broken down into different models:

### Components of a Notebook

- **NotebookMetadataModel**: Contains information about the notebook's kernel and language.
- **NotebookCellModel**: A base model representing different types of cells.
  - **NotebookCodeCellModel**: A subclass representing code cells with fields for source code and outputs.
  - **NotebookMarkdownCellModel**: A subclass representing markdown cells.
  - **NotebookRawCellModel**: A subclass representing raw cells.
- **NotebookOutputModel**: Represents the output produced by code cells.
- **NotebookFileModel**: The root model representing the entire notebook.

## Step 2: Code Implementation

Below is the code implementation that defines the models for handling the notebook structure.

In [ ]:
from dslmodel import DSLModel
from typing import List, Optional, Union
from pydantic import Field

# Notebook metadata model
class NotebookMetadataModel(DSLModel):
    kernelspec: dict = Field(..., description="Information about the notebook's kernel.")
    language_info: dict = Field(..., description="Information about the notebook's programming language.")

# Notebook output model for code cells
class NotebookOutputModel(DSLModel):
    output_type: str = Field(..., description="The type of the output (e.g., stream, display_data, error).")
    text: Optional[List[str]] = Field(None, description="Text output for streams or errors.")
    data: Optional[dict] = Field(None, description="Data output (e.g., images, JSON).")
    name: Optional[str] = Field(None, description="For stream output, the name (e.g., 'stdout', 'stderr').")
    execution_count: Optional[int] = Field(None, description="Execution count if relevant.")

# Base model for a notebook cell
class NotebookCellModel(DSLModel):
    cell_type: str = Field(..., description="The type of the cell (e.g., code, markdown, raw).")
    metadata: Optional[dict] = Field({}, description="Cell-specific metadata.")

# Code cell model
class NotebookCodeCellModel(NotebookCellModel):
    cell_type: str = "code"
    source: List[str] = Field(..., description="The source code inside the code cell.")
    execution_count: Optional[int] = Field(None, description="The execution count of the cell.")
    outputs: Optional[List[NotebookOutputModel]] = Field(None, description="Outputs produced by this code cell.")

# Markdown cell model
class NotebookMarkdownCellModel(NotebookCellModel):
    cell_type: str = "markdown"
    source: List[str] = Field(..., description="Markdown text inside the cell.")

# Raw cell model
class NotebookRawCellModel(NotebookCellModel):
    cell_type: str = "raw"
    source: List[str] = Field(..., description="Raw content inside the cell.")

# Root model for the entire notebook file
class NotebookFileModel(DSLModel):
    metadata: NotebookMetadataModel
    cells: List[Union[NotebookCodeCellModel, NotebookMarkdownCellModel, NotebookRawCellModel]]

    @classmethod
    def from_ipynb_file(cls, file_path: str) -> "NotebookFileModel":
        return cls.load(file_path, file_format="json")

    def to_ipynb_file(self, file_path: str) -> None:
        self.save(file_path, file_format="json")

## Step 3: Example Usage

Here's an example of how to use the `NotebookFileModel` to load, modify, and save a Jupyter notebook.

In [ ]:
# Load a notebook from a file
notebook = NotebookFileModel.from_ipynb_file("example_notebook.ipynb")

# Manipulate the notebook, e.g., add a new markdown cell
new_markdown_cell = NotebookMarkdownCellModel(source=["# New Title\n", "This is a new markdown cell."])
notebook.cells.append(new_markdown_cell)

# Save the modified notebook back to a file
notebook.to_ipynb_file("modified_notebook.ipynb")

In [ ]:
# Creating a new notebook programmatically
metadata = NotebookMetadataModel(
    kernelspec={"name": "python3", "display_name": "Python 3"},
    language_info={"name": "python", "version": "3.8"}
)

code_cell = NotebookCodeCellModel(source=["print('Hello, world!')"], execution_count=1)
markdown_cell = NotebookMarkdownCellModel(source=["# This is a markdown cell"])

notebook = NotebookFileModel(metadata=metadata, cells=[code_cell, markdown_cell])

# Serialize to an .ipynb file
notebook.to_ipynb_file("example_notebook.ipynb")


In [ ]:
from dslmodel import init_instant

init_instant()

hello_world = NotebookCodeCellModel.from_prompt("A code cell that prints 'Hello, world!' in Python")
print(hello_world)

In [ ]:
elementary_cellular_automaton = NotebookCodeCellModel.from_prompt("A code cell that implements the elementary cellular automaton rule 30")
print(elementary_cellular_automaton)

In [ ]:
from dslmodel import init_text

init_text()
elementary_cellular_automaton2 = NotebookCodeCellModel.from_prompt("A code cell that implements the elementary cellular automaton rule 30")
print(elementary_cellular_automaton2)